In [1]:
# State

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [195]:
# Case 1:
# true_object = get_z0_1q(c_sys)
# true_object.vec

# Case 2:
vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)

# Case 3:
# vec = np.array([1 / np.sqrt(2), 0, 0, 0], dtype=np.float64)
# true_object = State(c_sys, vec)

In [196]:
num_data = [100, 1000, 10000]
n_rep = 100

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")

  2%|▏         | 2/100 [00:00<00:08, 11.38it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


  2%|▏         | 2/100 [00:00<00:09, 10.71it/s]

elapsed_time:0.15124032894770303[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/state.py:283: ComplexWarning:

Casting complex values to real discards the imaginary part

  1%|          | 1/100 [00:00<00:15,  6.44it/s]

elapsed_time:0.19143440326054892[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


  1%|          | 1/100 [00:00<00:10,  9.47it/s]

elapsed_time:0.2047194480895996[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


100%|██████████| 100/100 [00:13<00:00,  7.30it/s]

elapsed_time:0.22844514846801758[min]



In [223]:
report.export_report("sample_qst.pdf",
    estimation_results_list,
    case_name_list,
    estimator_list,
    true_object,
    tester_objects,
    seed=seed
)

100%|██████████| 100/100 [00:00<00:00, 36996.60it/s]

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

/Users/tomoko/project/rcast/workspace/quara/quara/objects/state.py:283: ComplexWarning:

Casting complex values to real discards the imaginary part



​​Generating physicality violation test blocks ...


100%|██████████| 100/100 [00:00<00:00, 7550.50it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (sample_qst_MSE変更確認.pdf)


# Appendix

In [199]:
data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_name_list,
    true_object,
    show_analytical_results=True,
    estimator_list=estimator_list,
)

In [202]:
"LinearEstimator".replace("Estimator", "")

'Linear'

In [201]:
"LinearEstimator".rstrip("Linear")

'LinearEstimato'

In [159]:
case_names = case_name_list[:]

In [180]:
case_names = case_name_list[:]
data_dict = {}

for i, estimator in enumerate(estimator_list):
    estimator_name = estimator.__class__.__name__
    results = estimation_results_list[i]
    case_name = case_names[i]
    data = dict(estimation_results=results, case_name=case_name)

    if estimator_name in data_dict:
        data_dict[estimator_name]["estimation_results"].append(results)
        data_dict[estimator_name]["case_names"].append(case_name)
        data_dict[estimator_name]["estimators"].append(estimator)
    else:
        data_dict[estimator_name] = dict(
            estimation_results=[results], case_names=[case_name], estimators=[estimator]
        )

figs = []
for key, target_dict in data_dict.items():
    fig = data_analysis.make_mses_graph_estimation_results(
        target_dict["estimation_results"],
        target_dict["case_names"],
        true_object,
        additional_title_text=f"estimator={key}",
        show_analytical_results=True,
        estimator_list=target_dict["estimators"],
    )

    figs.append(fig)

In [182]:
figs = data_analysis.make_mses_graphs_estimator(estimation_results_list, case_name_list, true_object, estimator_list)

In [184]:
figs[1]

In [83]:
def make_mses_graphs_para(estimation_results_list:  List["EstimationResult"], case_names: List[str]) -> list:
    def _get_parameter(estimation_results: List["EstimationResult"]) -> bool:
        return estimation_results[0].qtomography.on_para_eq_constraint

    # Split data (True/False)
    true_dict = dict(title="True", estimation_results=[],  case_names=[])
    false_dict = dict(title="False", estimation_results=[],  case_names=[])

    for i, results in enumerate(estimation_results_list):
        if get_parameter(results):
            # on_para_eq_constraint = True
            true_dict["estimation_results"].append(results)
            true_dict["case_names"].append(case_names[i])
        else:
            # on_para_eq_constraint = False
            false_dict["estimation_results"].append(results)
            false_dict["case_names"].append(case_names[i])
    
    # Make figure
    figs = []
    for target_dict in [true_dict, false_dict]:
        fig = data_analysis.make_mses_graph_estimation_results(
            target_dict["estimation_results"],
            target_dict["case_names"],
            true_object,
            additional_title_text=f"parametrization={target_dict['title']}"
        )
        figs.append(fig)
    return figs

In [85]:
def _generate_figs_div(fig_info_list: List[List[dict]]) -> str:
    graph_block_html = ""
    for fig_info in fig_info_list:
        graph_subblock = (
                f"<div class='box'><img src={fig_info['image_path']}></div>"
        )
        graph_block_html += graph_subblock

    return graph_block_html

In [85]:
def _generate_figs_div(fig_info_list: List[List[dict]]) -> str:
    graph_block_html = ""
    for fig_info in fig_info_list:
        graph_subblock = (
                f"<div class='box'><img src={fig_info['image_path']}></div>"
        )
        graph_block_html += graph_subblock

    return graph_block_html

In [86]:

def _make_graphs_mses(
    func_make_graphs, mse_type: "str", **kwargs
) -> list:
    figs = func_make_graphs(**kwargs)
    fig_info_list = []

    for i, fig in enumerate(figs):
        fig_name = f"mse_type={mse_type}_{i}"
        fig.update_layout(width=_col2_fig_width, height=_col2_fig_height)
        path = _save_fig_to_tmp_dir(fig, fig_name)
        fig_info_list.append(dict(image_path=path, fig=fig, fig_name=fig_name))
    return fig_info_list

def _generate_figs_div(fig_info_list: List[List[dict]]) -> str:
    graph_block_html = ""
    for fig_info in fig_info_list:
        graph_subblock = (
                f"<div class='box'><img src={fig_info['image_path']}></div>"
        )
        graph_block_html += graph_subblock

    return graph_block_html

def generate_figs_div(func, **kwargs):
    fig_info_list = func(**kwargs)
    div_html = _generate_figs_div(fig_info_list)
    return div_html

In [87]:
_col2_fig_width = 500
_col2_fig_height = 400
from pathlib import Path
import tempfile

temp_dir_path = tempfile.mkdtemp()
_temp_dir_path = Path(temp_dir_path)


def _save_fig_to_tmp_dir(fig: "Figure", fig_name: str) -> str:
    dir_path = Path(_temp_dir_path)
    path = str(dir_path / f"{fig_name}.png")
    dir_path.mkdir(exist_ok=True)
    fig.write_image(path)

    return path
generate_figs_div(
    _make_graphs_mses,
    func_make_graphs=data_analysis.make_mses_graphs_estimator,
    mse_type="estimator",
    estimation_results_list=estimation_results_list,
    case_names=case_names,
    true_object=true_object,
    estimator_list=estimator_list
)

generate_figs_div(
    _make_graphs_mses,
    func_make_graphs=data_analysis.make_mses_graphs_para,
    estimation_results_list=estimation_results_list,
    case_names=case_names,
)

TypeError: _make_graphs_mses() missing 1 required positional argument: 'mse_type'

In [ ]:
figs = data_analysis.make_mses_graphs_estimator(estimation_results_list, case_name_list, true_object, estimator_list)

In [185]:
generate_figs_div(
    _make_graphs_mses,
    func_make_graphs=data_analysis.make_mses_graphs_estimator,
    mse_type="estimator",
    estimation_results_list=estimation_results_list,
    case_names=case_names,
    true_object=true_object,
    estimator_list=estimator_list
)

"<div class='box'><img src=/var/folders/ww/styyz2r10jb6dk8hl30y75nm0000gn/T/tmpmaryjkr5/mse_type=estimator_0.png></div><div class='box'><img src=/var/folders/ww/styyz2r10jb6dk8hl30y75nm0000gn/T/tmpmaryjkr5/mse_type=estimator_1.png></div>"

In [ ]:
for fig_info in fig_info_list:  # alpha
            graph_subblock = (
                f"<div class='box_col4'><img src={fig_info['image_path']}></div>"
            )
            graph_block_html += graph_subblock

        graph_block_html_all += graph_block_html

In [47]:
figs = make_mses_graphs_para(estimation_results_list[0], case_names)
figs[0].show()

In [28]:
# True / Falseで分ける方法
def get_parameter(estimation_results: List["EstimationResult"]) -> bool:
    return estimation_results[0].qtomography.on_para_eq_constraint

case_names = case_name_list[:]

true_dict = dict(title="True", estimation_results=[],  case_names=[])
false_dict = dict(title="False", estimation_results=[],  case_names=[])

for i, results in enumerate(estimation_results_list):
    if get_parameter(results):
        # on_para_eq_constraint = True
        true_dict["estimation_results"].append(results)
        true_dict["case_names"].append(case_names[i])
    else:
        # on_para_eq_constraint = False
        false_dict["estimation_results"].append(results)
        false_dict["case_names"].append(case_names[i])

In [37]:
figs = []
for target_dict in [true_dict, false_dict]:
    fig = data_analysis.make_mses_graph_estimation_results(
        target_dict["estimation_results"],
        target_dict["case_names"],
        true_object,
        additional_title_text=f"parametrization={target_dict['title']}"
    )
    figs.append(fig)

In [39]:
figs[1].show()

In [146]:
data_analysis.make_mses_graph_estimation_results(
    false_list["estimation_results"],
    false_list["case_names"],
    true_object
)

ValueError: 

In [148]:
fig = data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,  # 「EstimationResultのリスト」のリスト
    case_name_list,  # ケース名のリスト。ここに指定した文字列が凡例に表示される
    true_object,
        estimator_list=estimator_list
)
fig.show()

In [143]:
data_analysis.make_mses_graph_analytical(estimation_results_list, true_object, estimator_list)

True
False
